<a href="https://colab.research.google.com/github/RimgO/master/blob/main/Colab_de_WebCamIris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget   http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bunzip2 /content/shape_predictor_68_face_landmarks.dat.bz2
!rm /content/shape_predictor_68_face_landmarks.dat.bz2

--2021-12-27 15:57:31--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  23.9MB/s    in 2.6s    

2021-12-27 15:57:34 (23.9 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]

rm: cannot remove '/content/shape_predictor_68_face_landmarks.dat.bz2': No such file or directory


In [8]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import dlib
import numpy as np
import PIL
import io
import html
import time

In [9]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

  # function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def circle_to_bytes(circle_array):
  """
  Params:
          circle_array: Numpy array (pixels) containing circle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  circle_PIL = PIL.Image.fromarray(circle_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  circle_PIL.save(iobuf, format='png')
  # format return string
  circle_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return circle_bytes

In [10]:
# coding:utf-8

import os
import dlib
from imutils import face_utils
import cv2


class FaceLandmarkManager:
    """
    顔のランドマーク管理クラス
    """

    def __init__(self):
        """
        顔のランドマーク検出ツールの呼び出し
        """
        self._face_detector = dlib.get_frontal_face_detector()
        ##print('current_path:',os.getcwd())
        predictor_path = "/content" + os.sep + 'shape_predictor_68_face_landmarks.dat'
        self._face_predictor = dlib.shape_predictor(predictor_path)

        # 顔のランドマークリストの初期化
        self._face_landmark_list = []
        self._faces = []

    def clear_face_landmark_list(self):
        """
        顔のランドマークリストをリセット
        """
        self._face_landmark_list = []

    def detect_face_landmark(self, img):
        """
        画像から顔のランドマークを取得し保存
        :param img: 入力画像
        """

        # 顔検出
        img_gry = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        self._faces = self._face_detector(img_gry, 1)

        # 検出した全顔に対して処理
        for face in self._faces:
            # 顔のランドマーク検出
            landmark = self._face_predictor(img_gry, face)
            # 処理高速化のためランドマーク群をNumPy配列に変換(必須)
            landmark = face_utils.shape_to_np(landmark)
            self._face_landmark_list.append(landmark)


    def draw_face_landmark_list(self, img):
        """
        顔のランドマークを画像に描画
        :param img: 描画対象の画像
        """

        # 検出した顔全てにランドマークを描画
        for landmark in self._face_landmark_list:
            for (x, y) in landmark:
                cv2.circle(img, (x, y), 1, (255, 1, 1), -1)

    def get_face_landmark_list(self):
        """
        顔のランドマークリストの受け渡し
        :return self._face_landmark_list: 顔のランドマークリスト
        """
        return self._face_landmark_list


In [11]:
# Pタイル法用：眼球画像における虹彩の割合
IRIS_PER = 0.4


class EyeSystemManager:
    """
    目関連の処理の管理クラス
    """

    def __init__(self):
        # 目のランドマークの初期化
        self._eye_info = None

    def detect_eye_region(self, face_landmark):
        """
        顔のランドマークから目の領域を取得
        :param face_landmark:
        """
        # 顔のランドマークから両目の領域を格納後、リストへ追加
        eye_info = EyeRegionManager()
        eye_info.detect_eye_region(face_landmark)

        self._eye_info = eye_info

    @staticmethod
    def _detect_iris(eye_img):
        # グレースケール化後、ガウシアンフィルタによる平滑化
        eye_img_gry = cv2.cvtColor(eye_img, cv2.COLOR_BGR2GRAY)
        eye_img_gau = cv2.GaussianBlur(eye_img_gry, (5, 5), 0)

        # Pタイル法による2値化
        eye_img_thr = p_tile_threshold(eye_img_gau, IRIS_PER)

        cv2.rectangle(eye_img_thr, (0, 0), (eye_img_thr.shape[1] - 1, eye_img_thr.shape[0] - 1), (255, 255, 255), 1)

        # 輪郭抽出
        contours, hierarchy = cv2.findContours(eye_img_thr, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        # 輪郭から最小外接円により虹彩を求める
        iris = {'center': (0, 0), 'radius': 0}
        for i, cnt in enumerate(contours):
            (x, y), radius = cv2.minEnclosingCircle(cnt)
            center = (int(x), int(y))
            radius = int(radius)

            # 半径が大きすぎる場合、虹彩候補から除外
            if radius*0.3 < eye_img_thr.shape[0] < radius*0.8 :
                # # 虹彩候補の描画
                # cv2.circle(eye_img, center, radius, (255, 0, 0))
                continue

            # 最も半径が大きい円を虹彩と認定
            if iris['radius'] < radius:
                iris['center'] = center
                iris['radius'] = radius
                iris['num'] = i

        return iris

    def detect_iris_info(self, img):
        """
        両目の虹彩の取得
        :param img:
        :return:
        """

        # 眼球画像の取得
        self._eye_info.detect_eye_img(img)
        right_eye_img = self._eye_info.get_right_eye_img()
        left_eye_img = self._eye_info.get_left_eye_img()

        # 眼球画像から虹彩を抽出
        right_iris = self._detect_iris(right_eye_img)
        left_iris = self._detect_iris(left_eye_img)

        # 元画像における眼球座標と、眼球画像からの相対的な虹彩座標から、
        # 元画像における虹彩座標を計算
        right_eye_region = self._eye_info.get_right_eye_region()
        left_eye_region = self._eye_info.get_left_eye_region()

        right_center = (int(right_iris['center'][0] + right_eye_region['top_x']),
                        int(right_iris['center'][1] + right_eye_region['top_y']))
        left_center = (int(left_iris['center'][0] + left_eye_region['top_x']),
                       int(left_iris['center'][1] + left_eye_region['top_y']))

        right_iris['center'] = right_center
        left_iris['center'] = left_center

        return right_iris, left_iris

    def get_eye_region(self):
        """
        目領域の受け渡し
        :return self._eye_region: 目領域
        """
        return self._eye_info

In [12]:
class EyeRegionManager:
    """
    目領域を管理するクラス
    """

    def __init__(self):
        """
        初期化
        """
        self._right_eye_region = {}
        self._left_eye_region = {}
        self._right_eye_img = None
        self._left_eye_img = None
        self._right_eye_points = None
        self._left_eye_points = None

    def detect_eye_region(self, face_landmark):
        """
        ランドマークから目領域の座標を取得
        :param face_landmark:
        """
        # 右目切り出し
        self._right_eye_region = {'top_x': face_landmark[36][0], 'bottom_x': face_landmark[39][0],
                                  'top_y': face_landmark[37][1]
                                  if face_landmark[37][1] < face_landmark[38][1] else face_landmark[38][1],
                                  'bottom_y': face_landmark[41][1]
                                  if face_landmark[41][1] > face_landmark[40][1] else face_landmark[40][1]}

        # 左目切り出し
        self._left_eye_region = {'top_x': face_landmark[42][0], 'bottom_x': face_landmark[45][0],
                                 'top_y': face_landmark[43][1]
                                 if face_landmark[43][1] < face_landmark[45][1] else face_landmark[45][1],
                                 'bottom_y': face_landmark[47][1]
                                 if face_landmark[47][1] > face_landmark[46][1] else face_landmark[46][1]}

    def detect_eye_img(self, img):
        """
        目領域の座標から左右の目画像を取得
        :param img: 画像
        """
        self._right_eye_img = img[self._right_eye_region['top_y']:self._right_eye_region['bottom_y'],
                                  self._right_eye_region['top_x']:self._right_eye_region['bottom_x']]

        self._left_eye_img = img[self._left_eye_region['top_y']:self._left_eye_region['bottom_y'],
                                 self._left_eye_region['top_x']:self._left_eye_region['bottom_x']]

    def get_right_eye_region(self):
        """
        右目領域を受け渡す
        :return self._right_eye_region: 右目領域
        """
        return self._right_eye_region

    def get_left_eye_region(self):
        """
        左目領域を受け渡す
        :return self._left_eye_region: 左目領域
        """
        return self._left_eye_region

    def get_right_eye_img(self):
        """
        右目画像を受け渡す
        :return self._right_eye_img: 右目画像
        """
        return self._right_eye_img

    def get_left_eye_img(self):
        """
        左目画像を受け渡す
        :return self._left_eye_img: 左目画像
        """
        return self._left_eye_img

In [13]:
def p_tile_threshold(img_gry, per):
    """
    Pタイル法による2値化処理
    :param img_gry: 2値化対象のグレースケール画像
    :param per: 2値化対象が画像で占める割合
    :return img_thr: 2値化した画像
    """

    # ヒストグラム取得
    img_hist = cv2.calcHist([img_gry], [0], None, [256], [0, 256])

    # 2値化対象が画像で占める割合から画素数を計算
    all_pic = img_gry.shape[0] * img_gry.shape[1]
    pic_per = all_pic * per

    # Pタイル法による2値化のしきい値計算
    p_tile_thr = 0
    pic_sum = 0

    # 現在の輝度と輝度の合計(高い値順に足す)の計算
    for hist in img_hist:
        pic_sum += hist

        # 輝度の合計が定めた割合を超えた場合処理終了
        if pic_sum > pic_per:
            break

        p_tile_thr += 1

    # Pタイル法によって取得したしきい値で2値化処理
    ret, img_thr = cv2.threshold(img_gry, p_tile_thr, 255, cv2.THRESH_BINARY)

    return img_thr

In [14]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [15]:
from PIL import Image, ImageDraw

# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0 
face_manager = FaceLandmarkManager()

while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    img = js_to_image(js_reply["img"])

    # create transparent overlay for bounding box
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)

    # grayscale image for face detection
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    decimg = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # 顔のランドマークリストを取得
    face_manager.clear_face_landmark_list()
    face_manager.detect_face_landmark(decimg)
    face_landmark_list = face_manager.get_face_landmark_list()

    for face in face_manager._faces:
        (x, y, w, h) = face_utils.rect_to_bb(face)
        #print("face_bbox:",x,y,w,h)
    if 0:
        bbox_array = cv2.rectangle(bbox_array,(x,y),(x+w,y+h),(255,0,0),2)

    # 目領域の取得
    for face_landmark in face_landmark_list:

        eye_manager = EyeSystemManager()
        eye_manager.detect_eye_region(face_landmark)

        # 虹彩領域の取得
        right_iris, left_iris = eye_manager.detect_iris_info(decimg)

        # 虹彩領域の描画
        imgL = cv2.circle(bbox_array, right_iris['center'], right_iris['radius'], (0, 255, 0), 1)
        imgR = cv2.circle(bbox_array, left_iris['center'], left_iris['radius'], (0, 255, 0), 1)
    if 1:
      bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
      bbox_bytes = bbox_to_bytes(bbox_array)
      bbox = bbox_bytes


<IPython.core.display.Javascript object>

KeyboardInterrupt: ignored